# CHPT 4 - Sur-ajustement et sous-ajustement : identification et résolution

- outre l'utilisation de réseaux de neurones de capacité différentes (nombre de couche, taille des couches) deux méthodes sont couramment employées pour éviter le sur-ajustement
- l'ajout de régularisation sur les coefficients des couches (L1 ou L2) plus souvent L2
- l'ajout de dropout après les couches permettant d'éteindre aléatoirement des neuronesdans la couche précédentes (souvent entre 20 et 50 %)

## Chargement des données et préparation des jeux de données
Les données utilisées ici pour présenter ces différentes techniques sont issues de critique de films  du jeux de données IMDB (voir le [notebook](http://localhost:8888/lab/tree/CHPT3_DNN_IMDB.ipynb?file-browser-path=/) pour plus d'informations)

In [1]:
from keras.datasets import imdb
import numpy as np

(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words = 10000)

def vectorize_data(sequences,dimension=10000):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i,sequence]=1
    return results

x_train = vectorize_data(train_data)
y_train = np.asarray(train_labels).astype(float)

x_test = vectorize_data(test_data)
y_test = np.asarray(test_labels).astype(float)

/home/theo/anaconda3/envs/deeplwithpython/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/theo/anaconda3/envs/deeplwithpython/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


## Définition de plusieurs DNN

Définition de trois modèles de même capacitée (même architecture) mais avec différentes technique de traitement du sur-ajustement
- un modèle de base avec une couche cachée
- un modèle avec régularisation L2 sur les couches d'entrée et cachée
- un modèle avec dropout sur les couches d'entrée et cachée

In [2]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [3]:
from keras import layers, models, regularizers

# création du modèle originale
def build_model():
    """Retourne un réseau de neurones dense à trois couches"""
    model = models.Sequential()
    model.add(layers.Dense(16, activation = "relu",input_shape = (10000,)))
    model.add(layers.Dense(16, activation = "relu"))
    model.add(layers.Dense(1,activation = "sigmoid"))
    
    model.compile(optimizer = "rmsprop",
                  loss = "binary_crossentropy",
                  metrics = ["accuracy"])
    return model

# création du modèle de faible capacitéeLes
def build_model_fcap():
    """Retourne un réseau de neurones dense à trois couches"""
    model = models.Sequential()
    model.add(layers.Dense(4, activation = "relu",input_shape = (10000,)))
    model.add(layers.Dense(4, activation = "relu"))
    model.add(layers.Dense(1,activation = "sigmoid"))
    
    model.compile(optimizer = "rmsprop",
                  loss = "binary_crossentropy",
                  metrics = ["accuracy"])
    return model

# cration du modèle à forte capacitée
def build_model_hcap():
    """Retourne un réseau de neurones dense à trois couches"""
    model = models.Sequential()
    model.add(layers.Dense(16, activation = "relu",input_shape = (10000,)))
    model.add(layers.Dense(16, activation = "relu"))
    model.add(layers.Dense(1,activation = "sigmoid"))
    
    model.compile(optimizer = "rmsprop",
                  loss = "binary_crossentropy",
                  metrics = ["accuracy"])
    return model
# création d'un modèle avec pénalisation L2Les
# on peut utiliser aussi : regularizers.l1() ou regularizers.l1_l2()
def build_model_l2():
    model = models.Sequential()
    model.add(layers.Dense(16, activation = "relu",kernel_regularizer = regularizers.l2(0.001),input_shape = (10000,)))
    model.add(layers.Dense(16, activation = "relu",kernel_regularizer = regularizers.l2(0.001)))
    model.add(layers.Dense(1,activation = "sigmoid"))
    
    model.compile(optimizer = "rmsprop",
                  loss = "binary_crossentropy",
                  metrics = ["accuracy"])
    return model

# création d'un modèle avec dropout
def build_model_drop():
    model = models.Sequential()
    model.add(layers.Dense(16,activation="relu",input_shape = (10000,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(16,activation="relu", input_shape = (10000,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1,activation = "sigmoid"))
    
    model.compile(optimizer = "rmsprop",
                 loss = "binary_crossentropy",
                 metrics = ["accuracy"])
    return model
model_ori = build_model()
model_fcap = build_model_fcap()
model_hcap = build_model_hcap()
model_l2 = build_model_l2()
model_drop = build_model_drop()

list_model = [model_ori,model_fcap,model_hcap,model_l2,model_drop]

In [ ]:
epoch = 20
list_loss = np.zeros((epoch,len(list_model)))
for i,model in enumerate(list_model):
    model.fit(x_train,y_train,epochs=20,batch_size=512)
    list_loss[:,i] = model.history.history["loss"]